In [17]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta

In [18]:
# Sample Inputs
total_investment = 10000  # Total investment amount
start_date = "2024-05-01"  # Investment period start date
end_date = "2024-05-30"  # Investment period end date
end_date2 = "2024-05-28"  # Investment period end date

In [19]:
stocks = [
	{"Ticker": "ADANIPORTS.NS", "Weightage": 35.0000},
	{"Ticker": "APOLLOHOSP.NS", "Weightage": 30.0000},
	{"Ticker": "ASIANPAINT.NS", "Weightage": 35.0000},]


In [20]:
def calculate_rsi(data, periods=14):
	"""
	Calculates the Relative Strength Index (RSI) for a given DataFrame.

	Args:
		data: pandas DataFrame containing the 'Close' price data.
		periods: Number of periods for RSI calculation (default: 14).

	Returns:
		pandas Series containing the RSI values.
	"""

	delta = data["Close"].diff()
	gain = (delta.where(delta > 0, 0)).fillna(0)
	loss = (delta.where(delta < 0, 0)).fillna(0)

	avg_gain = gain.rolling(window=periods).mean()
	avg_loss = abs(loss.rolling(window=periods).mean())

	rs = avg_gain / avg_loss
	rsi = 100 - (100 / (1 + rs))

	return rsi

In [21]:
# Convert dates to Timestamps
start_date = pd.Timestamp(start_date)
end_date = pd.Timestamp(end_date)


# Function to adjust weightage by RSI
def adjust_weightage_by_rsi(weightage, rsi):
    if rsi is None or pd.isna(rsi):
        return weightage  # If RSI is missing, return the original weightage

    if rsi > 90:
        return weightage * 0.7  # Reduce by 30%
    elif rsi > 80:
        return weightage * 0.8  # Reduce by 20%
    elif rsi > 70:
        return weightage * 0.9  # Reduce by 10%
    elif rsi < 10:
        return weightage * 1.3  # Increase by 30%
    elif rsi < 20:
        return weightage * 1.2  # Increase by 20%
    elif rsi < 30:
        return weightage * 1.1  # Increase by 10%
    return weightage  # No change if between 30 and 70


# Results list
results = []

# Loop through each stock
for stock in stocks:
    ticker = stock["Ticker"]
    weightage = stock["Weightage"]

    # Download stock data
    data = yf.download(ticker, start=start_date, end=end_date)
    if data.empty:
        print(f"Data for {ticker} is not available. Skipping.")
        continue

    # Calculate RSI
    data["RSI"] = ta.rsi(data["Adj Close"], length=14)

    # Loop through each row of data
    # Loop through each row of data in filtered data
for index, row in data.iterrows():
    rsi = row["RSI"]
    close_price = row["Close"]

    # Ensure rsi is a scalar value
    if pd.isna(rsi):
        rsi = None

    # Adjust weightage dynamically
    adjusted_weightage = adjust_weightage_by_rsi(weightage, rsi)

    # Calculate allocated investment and shares
    allocated_investment = adjusted_weightage * total_investment
    shares = allocated_investment // close_price if close_price > 0 else 0

    # Append results
    results.append(
        {
            "Ticker": ticker,
            "Date": index.date(),
            "Weightage": weightage,
            "Adjusted Weightage": adjusted_weightage,
            "Allocated Investment": allocated_investment,
            "Close Price": close_price,
            "RSI": rsi,
            "Shares": shares,
        }
    )


# Convert results to DataFrame
summary = pd.DataFrame(results)

# Reorder and display columns
summary = summary[
    [
        "Ticker",
        "Date",
        "Weightage",
        "Adjusted Weightage",
        "Allocated Investment",
        "Close Price",
        "RSI",
        "Shares",
    ]
]

# Save the summary to CSV
summary.to_csv("investment_summary.csv", index=False)
print("\nSummary saved to 'investment_summary.csv'")

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().